In [1]:
import numpy as np

In [2]:
#Load wires
wires = np.genfromtxt('day3_input.txt', dtype=np.str)
wire0 = wires[0].split(',')
for i in range(0, len(wire0)):
    wire0[i] = [wire0[i][0], np.int(wire0[i][1:])]
wire1 = wires[1].split(',')
for i in range(0, len(wire1)):
    wire1[i] = [wire1[i][0], np.int(wire1[i][1:])]

In [3]:
def verticies(wire):
    '''Define verticies using wire routes
       Set:
       Origin = [0,0]
       U/D is +/- x
       R/L is +/- y'''
    vertex = [[0,0]]
    for inst in wire:
        x = vertex[-1][0]
        y = vertex[-1][1]
        if inst[0] == 'U':
            y += inst[1]
        elif inst[0] == 'D':
            y -= inst[1]
        elif inst[0] == 'R':
            x += inst[1]
        elif inst[0] == 'L':
            x -= inst[1]
        else:
            print('FAILED:', inst[0], inst[1])
            break
        vertex.append([x,y])
    return np.array(vertex)

In [4]:
def intersect(vert0, vert1):
    '''Find intersection of wires'''
    intersect = []
    manhattan = []
    for i in range(0, len(vert0)-1):
        for j in range(0, len(vert1)-1):
            #If wires are parallel they cannot cross
            if vert0[i][0] == vert0[i+1][0] and vert1[j][0] == vert1[j+1][0]:
                #parallel in x
                continue
            elif vert0[i][1] == vert0[i+1][1] and vert1[j][1] == vert1[j+1][1]:
                #parallel in y
                continue
                
            #Wire 0 is lying along y. Wire 1 is lying along x
            if ((vert1[j][0] < vert0[i][0] and vert1[j+1][0] > vert0[i][0]) or\
                (vert1[j][0] > vert0[i][0] and vert1[j+1][0] < vert0[i][0])) and\
               ((vert1[j][1] < vert0[i][1] and vert1[j][1] > vert0[i+1][1]) or\
                (vert1[j][1] > vert0[i][1] and vert1[j][1] < vert0[i+1][1])):
                #between x and crosses y
                x = vert0[i][0]
                y = vert1[j][1]
                
            #Wire 0 is lying along x. Wire 1 is lying along y
            elif ((vert1[j][1] < vert0[i][1] and vert1[j+1][1] > vert0[i][1]) or\
                  (vert1[j][1] > vert0[i][1] and vert1[j+1][1] < vert0[i][1])) and\
                 ((vert1[j][0] < vert0[i][0] and vert1[j][0] > vert0[i+1][0]) or\
                  (vert1[j][0] > vert0[i][0] and vert1[j][0] < vert0[i+1][0])):
                #crosses x and between y
                x = vert1[j][0]
                y = vert0[i][1]
                
            else:
                continue
            
            intersect.append([x,y])
            manhattan.append(abs(x)+abs(y))
    return np.array(intersect), np.array(manhattan)

In [5]:
#Find corners of wires
vertex0 = verticies(wire0)
vertex1 = verticies(wire1)

#Find intersection points and Manhatten distance
cross, man = intersect(vertex0, vertex1)
print('Part 1 solution:', np.min(man))

Part 1 solution: 399


In [6]:
def distance(wire, inter):
    '''Find distance to position on wire to intersections
       Brute force by following the wire and checking every position
       Assumes inter are unordered
       Set:
       Origin = [0,0]
       U/D is +/- x
       R/L is +/- y'''
    #Want blank distances, -ve so we can identify them easily
    distances = np.ones(len(inter), dtype=np.int)
    distances *= -1
    x = 0
    y = 0
    dist = 0
    done = len(inter)
    
    for inst in wire:
        drctn_x = 0
        drctn_y = 0
        if inst[0] == 'U':
            drctn_y = 1
        elif inst[0] == 'D':
            drctn_y = -1
        elif inst[0] == 'R':
            drctn_x = 1
        elif inst[0] == 'L':
            drctn_x = -1
        else:
            print('FAILED:', inst[0], inst[1])
            break
            
        for i in range(0, inst[1]):
            #Move along wire
            x += drctn_x
            y += drctn_y
            dist += 1
            
            #Check if we match an intersection
            x_match = np.where(inter[:,0] == x)[0]
            if len(x_match) > 0:
                y_match = np.where(inter[:,1] == y)[0]
                if len(y_match) > 0:
                    match = np.intersect1d(x_match, y_match)
                    if len(match) > 0:
                        #Only update distance if no distance has been found
                        if distances[match[0]] < 0:
                            distances[match[0]] = dist
                            done -= 1
            if done == 0:
                break
        if done == 0:
            break
    return distances

In [7]:
#Find wire distances
dist0 = distance(wire0, cross)
dist1 = distance(wire1, cross)

dist = dist0+dist1
print('Part 2 solution:', np.min(dist))

Part 2 solution: 15678
